In [11]:
import nbimporter
import StockNews2018Pkg as SP
import threading

#Preferences:
ConsiderAt = 0.5
BuyVShort = 0.9
#InterestedStocks = ["INTC","NVDA","TWTR","AMZN","SNAP","IBM","WMT","MSFT","WMT","PYPL","AAPL","QCOM","TSLA","FB","BABA","GS","GOOG","MMM","AXP","BA","CAT","CVX","CSCO","KO","DIS","XOM","GE","HD","INTC","JNJ","JPM","MCD","MRK","NKE","PFE","PG","TRV","UTX","UNH","VZ","V"]
InterestedStocks = ["INTC","NVDA","TWTR","SNAP","IBM","WMT","MSFT","WMT","PYPL","AAPL","QCOM","TSLA","FB","BABA","GS","GOOG","MMM","AXP","BA","CAT","CVX","CSCO","KO","DIS","XOM","GE","HD","INTC"]
verbosity = 0

#Folder Stuff:
import os
import glob
from datetime import datetime

#Round Down
import math

#CSV Stuff:
import pandas as pd
import csv

#Investopedia API:
import ita as Investopedia
client = Investopedia.Account("kennebrookes123@gmail.com", "albatross4321")

In [7]:
def UpdateHoldings():
    TopStocks = Analyze(InterestedStocks)
    NewAcct = NewAccount()
    
    if (NewAcct == False):
        list_of_files = glob.glob('./RealTimeHoldings2018/*') # * means all if need specific format then *.csv
        latest_file = max(list_of_files, key=os.path.getctime)
        lastholdings = pd.read_csv(latest_file)
    
    CurrentTime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    #NEWHOLDINGS:
    with open('RealTimeHoldings2018/' + CurrentTime + '.csv', 'a') as csvfile: #wb
        filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(["Ticker", "BuyPrice", "Amount"])
        
        MaintainedStocks = []
        #Sell/Maintain The Old Stocks:
        if (NewAcct == False):
            for i in range(0,lastholdings.shape[0]):
                stockname = lastholdings.iloc[i,0]
                buyprice = lastholdings.iloc[i,1]
                amt = int(lastholdings.iloc[i,2])
                currentprice = (float) Investopedia.get_quote(stockname)
                if (not (stockname in TopStocks)) and (currentprice > buyprice):
                    #client.trade(stockname, Investopedia.Action.sell, amt)
                    print("SELLING: " + stockname)
                else:
                    filewriter.writerow([stockname, buyprice, amt]) #Keep if either (1) still recommended or (2) lower
                    MaintainedStocks.append(stockname)
        
        ToBuy = []
        #Filter out Tops:
        if (NewAcct == False):
            for y in range(0,len(TopStocks) - 1):
                if not ((TopStocks[y]) in MaintainedStocks):
                    ToBuy.append(TopStocks[y])
        else: #If it is a new account:
            ToBuy = TopStocks
        
        #Buying Power
        status = client.get_portfolio_status()
        BuyBal = status.buying_power
        
        if (verbosity == 1):
            print("About To Buy: ")
            print(ToBuy)
        
        #Buy
        for b in range(0,len(ToBuy)):
            ordername = ToBuy[b]
            orderprice = Investopedia.get_quote(ordername)
            orderamount = int(math.floor(((BuyBal*BuyVShort)/(len(ToBuy)))/orderprice))
            print("_"+ordername+"_")
            print("_"+str(orderamount)+"_")
            client.trade(str(ordername), Investopedia.Action.buy, int(orderamount))
            print("BUYING " + ordername)
            filewriter.writerow([ordername, orderprice, orderamount])
                    
                
    threading.Timer(300.0, UpdateHoldings).start()
    

SyntaxError: invalid syntax (<ipython-input-7-ce6097a2e270>, line 25)

In [8]:
def Analyze(stocklist):
    resultlist = []
    for i in range(0,len(stocklist)):
        score = SP.Predict(stocklist[i],0)
        resultlist.append(str("%.2f" % score) + " " + stocklist[i])
        
    TopStocksNames = []
    for y in range(0,len(resultlist)):
        if (float(resultlist[y][:4]) > ConsiderAt):
            TopStocksNames.append(resultlist[y][5:]) #Add name
            
    return TopStocksNames

In [9]:
def NewAccount(): 
    path, dirs, files = os.walk("./RealTimeHoldings2018/").next()
    file_count = len(files)
    if (file_count == 0):
        New = True
    else:
        New = False
        
    return New

In [12]:
#reload(SP)
UpdateHoldings()

Found 23 Articles for Stock INTC
Collected data from INTC for 4654 days!
20 instances found
Loaded model from disk
[[ 1.        ]
 [ 0.99869925]
 [ 0.99998355]
 [ 0.9999795 ]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]
 [ 0.99872637]
 [ 0.98452151]
 [ 0.02068773]
 [ 1.        ]
 [ 0.99992424]
 [ 0.97265112]
 [ 0.28525347]
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]
 [ 0.95354855]
 [ 1.        ]]
19
INTC is Predicted to RISE. +16,-2
Found 18 Articles for Stock NVDA
Collected data from NVDA for 4654 days!
17 instances found
Loaded model from disk
[[ 0.98603916]
 [ 0.9757731 ]
 [ 0.99996513]
 [ 0.69372028]
 [ 0.99999708]
 [ 1.        ]
 [ 0.781084  ]
 [ 0.20809759]
 [ 1.        ]
 [ 0.80110961]
 [ 0.99315643]
 [ 0.99999928]
 [ 0.99999785]
 [ 0.86972529]
 [ 0.00168392]
 [ 1.        ]]
16
NVDA is Predicted to RISE. +13,-2
Found 21 Articles for Stock TWTR
Collected data from TWTR for 1170 days!
18 instances found
Loaded model from disk
[[ 1.        ]
 [ 0.2861273 ]
 [ 0.00345291]
 [ 0.7182

Traceback (most recent call last):
  File "/Users/ethanshaotran/anaconda/lib/python2.7/site-packages/newspaper/parsers.py", line 54, in fromstring
    cls.doc = lxml.html.fromstring(html)
  File "/Users/ethanshaotran/anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 876, in fromstring
    doc = document_fromstring(html, parser=parser, base_url=base_url, **kw)
  File "/Users/ethanshaotran/anaconda/lib/python2.7/site-packages/lxml/html/__init__.py", line 765, in document_fromstring
    "Document is empty")
ParserError: Document is empty


15 instances found
Loaded model from disk
[[ 1.        ]
 [ 0.99998522]
 [ 0.99996382]
 [ 0.29566664]
 [ 0.92774081]
 [ 0.04262183]
 [ 0.00600957]
 [ 0.0015913 ]
 [ 0.00568701]
 [ 1.        ]
 [ 0.8331874 ]
 [ 0.05820321]
 [ 1.        ]
 [ 0.9999997 ]]
14
FB is Predicted to RISE. +7,-6
Found 16 Articles for Stock BABA
Collected data from BABA for 953 days!
12 instances found
Loaded model from disk
[[ 0.99995732]
 [ 0.99997866]
 [ 0.71741062]
 [ 0.0986665 ]
 [ 0.11368801]
 [ 0.99933875]
 [ 0.99999946]
 [ 0.35423145]
 [ 1.        ]
 [ 0.99025863]
 [ 0.99213427]]
11
BABA is Predicted to RISE. +7,-3
Found 22 Articles for Stock GS
Collected data from GS for 4654 days!
16 instances found
Loaded model from disk
[[ 1.        ]
 [ 1.        ]
 [ 0.88013709]
 [ 1.        ]
 [ 0.66749072]
 [ 0.95819503]
 [ 0.00277195]
 [ 0.06936634]
 [ 1.        ]
 [ 1.        ]
 [ 0.99999344]
 [ 0.99997836]
 [ 0.99998957]
 [ 1.        ]
 [ 0.89609355]]
15
GS is Predicted to RISE. +12,-2
Found 21 Articles for Sto

ValueError: invalid literal for float(): 1,127.46